In [1]:
%load_ext autoreload
%autoreload 2
from icu_pipeline.pipeline import (
    Pipeline,
    DataSource,
    SourceConfig,
)
from pathlib import Path
from dotenv import load_dotenv
import os
import pandas as pd


load_dotenv()

POSTGRES_USER = os.getenv("POSTGRES_USER")
POSTGRES_PASSWORD = os.getenv("POSTGRES_PASSWORD")
POSTGRES_HOST = os.getenv("POSTGRES_HOST")
POSTGRES_PORT = os.getenv("POSTGRES_PORT")
MIMIC_DB = os.getenv("MIMIC_DB")
AMDS_DB = os.getenv("AMDS_DB")
EICU_DB = os.getenv("EICU_DB")

/home/fu/anaconda3/envs/oicu/lib/python3.12/site-packages/pydantic/_internal/_fields.py:200: UserWarning: Field name "schema" in "ConceptConfig" shadows an attribute in parent "BaseModel"
  warnings.warn(
INFO:root:New Logger Created


In [2]:
mimic_config = SourceConfig(
    f"postgresql+psycopg://{POSTGRES_USER}:{POSTGRES_PASSWORD}@{POSTGRES_HOST}/{MIMIC_DB}",
    limit=500,
    chunksize=100
)

In [3]:
from icu_pipeline.sink.pandas import PandasSink

pipeline = Pipeline(
    source_configs={
        DataSource.MIMICIV: mimic_config,
    },
    sink_mapper=PandasSink(),
)

concepts = ["HeartRate", "SystolicBloodPressure"] # "HeartRate", "SystolicBloodPressure"

In [4]:
result = pipeline.transform(concepts)
print(pipeline._graph)

Pipe Class: InMemoryPipe
Node Class: InMemoryNode


In [5]:
result = [{
    k: v for k,v in c.items()
} for c in result]
out = {}
for c in pipeline._sink_mapper._sources:
    out[c] = pd.concat([v[c] for v in result])
out

DEBUG:psycopg.pq:couldn't import psycopg 'c' implementation: No module named 'psycopg_c'
DEBUG:root:Added 0 new dependencies to the Graph.


Pipe Class: InMemoryPipe
Node Class: InMemoryNode
Concept(1,HeartRate)-->FrequencyConverter(3)-->PandasSink(0)
Concept(2,SystolicBloodPressure)-->PressureConverter(4)-->PandasSink(0)


DEBUG:root:
            SELECT DISTINCT subject_id
            FROM "mimiciv_icu"."icustays"
            LIMIT 500
        
DEBUG:root:
            SELECT "subject_id" AS "patient_id", "charttime" AS "timestamp", "valuenum" AS "value", "valueuom" AS "unit"
            FROM "mimiciv_icu"."chartevents"
            WHERE "itemid" = '220045'
            AND subject_id IN (10000032,10000980,10001217,10001725,10001884,10002013,10002155,10002348,10002428,10002430,10002443,10002495,10002760,10002930,10003019,10003046,10003400,10003502,10004113,10004235,10004401,10004422,10004457,10004606,10004720,10004733,10004764,10005123,10005348,10005606,10005817,10005866,10005909,10006053,10006131,10006277,10006580,10006821,10007058,10007795,10007818,10007920,10007928,10008077,10008100,10008287,10008454,10008924,10009035,10009049,10009628,10009686,10010058,10010471,10010867,10011189,10011365,10011398,10011427,10011668,10011938,10012055,10012206,10012292,10012438,10012476,10012552,10012853,10013015,10013049

{'HeartRate':                                           subject       effective_date_time  \
 0      {'reference': '10000032', 'type': 'mimic'} 2180-07-23 22:00:00+00:00   
 1      {'reference': '10000032', 'type': 'mimic'} 2180-07-23 19:00:00+00:00   
 2      {'reference': '10000032', 'type': 'mimic'} 2180-07-23 20:00:00+00:00   
 3      {'reference': '10000032', 'type': 'mimic'} 2180-07-23 21:00:00+00:00   
 4      {'reference': '10000032', 'type': 'mimic'} 2180-07-23 14:12:00+00:00   
 ...                                           ...                       ...   
 11769  {'reference': '10107317', 'type': 'mimic'} 2153-04-22 13:00:00+00:00   
 11770  {'reference': '10107317', 'type': 'mimic'} 2153-04-22 14:00:00+00:00   
 11771  {'reference': '10107317', 'type': 'mimic'} 2153-04-22 15:00:00+00:00   
 11772  {'reference': '10107317', 'type': 'mimic'} 2153-04-22 16:00:00+00:00   
 11773  {'reference': '10107317', 'type': 'mimic'} 2153-04-22 17:00:00+00:00   
 
                         

In [6]:
print(pipeline._graph)

Pipe Class: InMemoryPipe
Node Class: InMemoryNode
Concept(1,HeartRate)-->FrequencyConverter(3)-->PandasSink(0)
Concept(2,SystolicBloodPressure)-->PressureConverter(4)-->PandasSink(0)


In [7]:
pipeline._sink_mapper._sources

{'HeartRate': <icu_pipeline.graph.in_memory.InMemoryPipe at 0x7513cc09e3c0>,
 'SystolicBloodPressure': <icu_pipeline.graph.in_memory.InMemoryPipe at 0x7513cc09e120>}

In [8]:
out["HeartRate"]

,subject,effective_date_time,value_quantity,code
0,"{'reference': '10000032', 'type': 'mimic'}",2180-07-23 22:00:00+00:00,"{'value': 94.0, 'unit': 'bpm'}","{'coding': {'code': 'HeartRate', 'system': 'sn..."
1,"{'reference': '10000032', 'type': 'mimic'}",2180-07-23 19:00:00+00:00,"{'value': 97.0, 'unit': 'bpm'}","{'coding': {'code': 'HeartRate', 'system': 'sn..."
2,"{'reference': '10000032', 'type': 'mimic'}",2180-07-23 20:00:00+00:00,"{'value': 100.0, 'unit': 'bpm'}","{'coding': {'code': 'HeartRate', 'system': 'sn..."
3,"{'reference': '10000032', 'type': 'mimic'}",2180-07-23 21:00:00+00:00,"{'value': 94.0, 'unit': 'bpm'}","{'coding': {'code': 'HeartRate', 'system': 'sn..."
4,"{'reference': '10000032', 'type': 'mimic'}",2180-07-23 14:12:00+00:00,"{'value': 91.0, 'unit': 'bpm'}","{'coding': {'code': 'HeartRate', 'system': 'sn..."
...,...,...,...,...
11769,"{'reference': '10107317', 'type': 'mimic'}",2153-04-22 13:00:00+00:00,"{'value': 86.0, 'unit': 'bpm'}","{'coding': {'code': 'HeartRate', 'system': 'sn..."
11770,"{'reference': '10107317', 'type': 'mimic'}",2153-04-22 14:00:00+00:00,"{'value': 99.0, 'unit': 'bpm'}","{'coding': {'code': 'HeartRate', 'system': 'sn..."
11771,"{'reference': '10107317', 'type': 'mimic'}",2153-04-22 15:00:00+00:00,"{'value': 80.0, 'unit': 'bpm'}","{'coding': {'code': 'HeartRate', 'system': 'sn..."
11772,"{'reference': '10107317', 'type': 'mimic'}",2153-04-22 16:00:00+00:00,"{'value': 81.0, 'unit': 'bpm'}","{'coding': {'code': 'HeartRate', 'system': 'sn..."


In [9]:
from icu_pipeline.graph import Node,Pipe
from icu_pipeline.graph.base import Graph

g = Graph()
a = Node("1")
b = Node("2")
c = Node("3")
d = Node("4")
g.addPipe(a,b)
g.addPipe(b,c)
g.addPipe(d,c)
print(g)
g.addPipe(b,a)

Pipe Class: InMemoryPipe
Node Class: InMemoryNode
InMemoryNode(5)-->InMemoryNode(6)-->InMemoryNode(7)
InMemoryNode(8)-->InMemoryNode(7)


AssertionError: Graph has Circular dependencies!